In [1]:
from process_config import *
from rime_functions import *
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler
from dask.distributed import Client
import glob
import numpy as np
import pandas as pd
import pyam
import re
import time
import xarray as xr
import dask

<IPython.core.display.Javascript object>

# Overview

**Inputs:**
 - Emissions or GMT scenarios from IAM.  
 - Climate impacts database aggregated to region (e.g. country or R10) by Global Warming Level.  

**Outputs:**
 - Table data of climate impacts through time in IAMC format, per scenario.

Steps  
1. Import climate impacts database. NetCDF 4D file with different impacts as variables.
2. Import the scenarios to be run. Can be global mean temp scenarios or Emissions|CO2 scenarios.
3. Preprocessing steps.
4. Run.


### Import the climate impacts database files


In [2]:
filesall = glob.glob(fname_input_climate)
files = filesall[12:14]
f = files[0]
# load input climate impacts data file
ds = xr.open_mfdataset(f)
ds = ds.sel(year=years)
ds

<xarray.Dataset>
Dimensions:                                     (gmt: 251, year: 9, ssp: 3,
                                                 region: 226)
Coordinates:
  * gmt                                         (gmt) float64 1.2 1.206 ... 3.5
  * year                                        (year) int64 2015 2025 ... 2095
  * ssp                                         (ssp) object 'SSP1' ... 'SSP3'
  * region                                      (region) object 'AFG' ... 'wo...
Data variables:
    sdii|Exposure|Land area                     (gmt, year, ssp, region) float64 dask.array<chunksize=(251, 9, 3, 226), meta=np.ndarray>
    sdii|Exposure|Population                    (gmt, year, ssp, region) float64 dask.array<chunksize=(251, 9, 3, 226), meta=np.ndarray>
    sdii|Exposure|Population|%                  (gmt, year, ssp, region) float64 dask.array<chunksize=(251, 9, 3, 226), meta=np.ndarray>
    sdii|Hazard|Absolute|Land area weighted     (gmt, year, ssp, region) float64 dask.array<chunksize=(251, 9, 3, 226), meta=np.ndarray>
    sdii|Hazard|Absolute|Population weighted    (gmt, year, ssp, region) float64 dask.array<chunksize=(251, 9, 3, 226), meta=np.ndarray>
    sdii|Hazard|Risk score|Population weighted  (gmt, year, ssp, region) float64 dask.array<chunksize=(251, 9, 3, 226), meta=np.ndarray>

In [3]:
varis = list(ds.data_vars.keys())[:lvaris]
dsi = ds[varis]
print(f"# of variables = {len(varis)}")


# of variables = 6


### Import scenarios data

Load an IAMC scenarios dataset.  
Decide whether using global mean temperature or CO2 mode.  
Assign SSPs if missing and fix duplicate temperatures.  


In [4]:
df_scens_in = pyam.IamDataFrame(fname_input_scenarios)
# SCENARIO DATA PRE-PROCESSING
# Filter for temperature variable

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading file C:\Users\byers\IIASA\ECE.prog - Documents\Research Theme - NEXUS\Hotspots_Explorer_2p0\rcre_testing\testing_2\emissions_temp_AR6_small.xlsx
pyam.core - INFO: Reading meta indicators


In [5]:
df_scens_in

<class 'pyam.core.IamDataFrame'>
Index:
 * model    : AIM/CGE 2.0, AIM/CGE 2.1, AIM/CGE 2.2, ... WITCH-GLOBIOM 4.4 (44)
 * scenario : 1.5C, 2.5C, 2C, 2CNow_Gradual, ... WB2C (559)
Timeseries data coordinates:
   region   : World (1)
   variable : AR6 climate diagnostics|Infilled|Emissions|CO2, ... (3)
   unit     : K, Mt CO2/yr (2)
   year     : 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, ... 2100 (86)
Meta indicators:
   Category (object) C3, C5, C6, C7, C4, C1, C2, C8 (8)
   Category_name (object) C3: limit warming to 2°C (>67%), ... (8)
   Category_subset (object) C3y_+veGHGs, C5, C6, C7, C4, C1a_NZGHGs, ... C8 (10)
   Subset_Ch4 (object) Limit to 2C (>67%) immediate 2020 action, ... (6)
   Category_Vetting_historical (object) C3, C5, C6, C7, C4, C1, C2, C8 (8)
   ...

In [12]:
mode = 'CO2'
if mode == "GMT":
    dfp = df_scens_in.filter(variable=temp_variable)
elif mode == "CO2":
    dfp = prepare_cumCO2(df_scens_in, years=years, use_dask=True)
    ts = dfp.timeseries().apply(co2togmt_simple)
    ts = pyam.IamDataFrame(ts)
    ts.rename(
        {
            "variable": {ts.variable[0]: "RIME|GSAT_tcre"},
            "unit": {ts.unit[0]: "°C"},
        },
        inplace=True,
    )
    # Export data to check error and relationships
    # ts.append(dfp).to_csv('c://users//byers//downloads//tcre_gmt_output.csv')
    dfp = ts
    dfp.meta = df_scens_in.meta.copy()
dfp = dfp.filter(year=years)

In [14]:
dfp

<class 'pyam.core.IamDataFrame'>
Index:
 * model      : AIM/CGE 2.1, AIM/CGE 2.2, AIM/Hub-Global 2.0, ... WITCH-GLOBIOM 4.4 (34)
 * scenario   : 1.5C, 2.5C, 2C, 2CNow_Gradual, ... WB2C (511)
Timeseries data coordinates:
   region     : World (1)
   variable   : RIME|GSAT_tcre (1)
   unit       : °C (1)
   year       : 2015, 2025, 2035, 2045, 2055, 2065, 2075, 2085, 2095 (9)
   ssp_family : SSP1, SSP2, SSP3, SSP4, SSP5 (5)
Meta indicators:
   exclude (bool) False (1)

In [13]:
few_scenarios = False
very_few_scenarios = False

if few_scenarios:
    dfp = dfp.filter(Category=["C1*"])
    if very_few_scenarios:
        dfp = dfp.filter(model="REMIND 2.1*", scenario="*") 
        
        
#  Assign SSP to meta and select SSP2 in case SSP not present in name
dfp = ssp_helper(dfp, ssp_meta_col="Ssp_family", default_ssp="SSP2")

# Fix duplicate temperatures
dft = dfp.timeseries().reset_index()
dft = dft.apply(fix_duplicate_temps, years=years, axis=1)
dft.reset_index(inplace=True, drop=True)
dft

,model,scenario,region,variable,unit,ssp_family,2015,2025,2035,2045,2055,2065,2075,2085,2095
0,AIM/CGE 2.1,CD-LINKS_INDC2030i_1600,World,RIME|GSAT_tcre,°C,SSP2,1.321,1.516,1.683,1.778,1.842,1.881,1.915000,1.955,1.996
1,AIM/CGE 2.1,CD-LINKS_INDCi,World,RIME|GSAT_tcre,°C,SSP2,1.321,1.517,1.706,1.902,2.104,2.317,2.542000,2.780,3.033
2,AIM/CGE 2.1,CD-LINKS_NDC2030i_1000,World,RIME|GSAT_tcre,°C,SSP2,1.321,1.516,1.683,1.764,1.784,1.773,1.755000,1.741,1.728
3,AIM/CGE 2.1,CD-LINKS_NPi2020_1000,World,RIME|GSAT_tcre,°C,SSP2,1.321,1.505,1.619,1.678,1.712,1.718,1.714148,1.722,1.727
4,AIM/CGE 2.1,CD-LINKS_NPi2020_1600,World,RIME|GSAT_tcre,°C,SSP2,1.321,1.509,1.636,1.734,1.813,1.868,1.919000,1.976,2.034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,WITCH-GLOBIOM 4.4,CD-LINKS_NPi,World,RIME|GSAT_tcre,°C,SSP2,1.322,1.535,1.796,2.096,2.426,2.784,3.169000,3.579,3.998
989,WITCH-GLOBIOM 4.4,CD-LINKS_NPi2020_1000,World,RIME|GSAT_tcre,°C,SSP2,1.322,1.507,1.616,1.693,1.744,1.769,1.777000,1.766,1.737
990,WITCH-GLOBIOM 4.4,CD-LINKS_NPi2020_1600,World,RIME|GSAT_tcre,°C,SSP2,1.322,1.516,1.662,1.773,1.864,1.935,1.989000,2.018,2.025
991,WITCH-GLOBIOM 4.4,CD-LINKS_NPi2020_400,World,RIME|GSAT_tcre,°C,SSP2,1.322,1.503,1.584,1.621,1.629,1.614,1.581000,1.524,1.451


### START PROCESSING

In [8]:
start = time.time()
year_res = 10
parallel = True
if parallel:
    """
    For parallel processing, convert dft as a wide IAMC pd.Dataframe
    into a dask.DataFrame.
    """
    ddf = dd.from_pandas(dft, npartitions=1000)

    # dfx = dft.iloc[0].squeeze()  # FOR DEBUIGGING THE FUNCTION
    outd = ddf.apply(
        calculate_impacts_gmt, dsi=dsi, axis=1, meta=("result", None)
    )

    with ProgressBar():
        # try:
        df_new = outd.compute(num_workers=num_workers)
else:
    df_new = dft.apply(calculate_impacts_gmt, dsi=dsi, axis=1)

expandeddGMT = pd.concat([df_new[x] for x in df_new.index])
print(f" Done:  {time.time()-start}")

filename = f"RIME_output_{region}_{year_res}yr.csv"

# expandedd.to_csv(filename, encoding="utf-8", index=False)
print(f" Saved: {region} yrs={year_res}\n  {time.time()-start}")
print(f"{len(dsi.data_vars)} variables, {len(dfp.meta)} scenarios")

[                                        ] | 0% Completed | 124.97 ms

C:\Users\byers\Anaconda3\envs\py310\lib\site-packages\dask\dataframe\utils.py:291: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series([], dtype=dtype, name=name, index=index)


[########################################] | 100% Completed | 105.32 s
 Done:  105.48372197151184
 Saved: COUNTRIES yrs=10
  105.48372197151184
6 variables, 1202 scenarios


In [11]:
expandeddGMT

,model,scenario,region,variable,unit,2015,2025,2035,2045,2055,2065,2075,2085,2095
0,AIM/CGE 2.0,SSP1-26,AFG,RIME|sdii|Exposure|Land area,km2,0.000000,0.000000,445.506108,954.655946,1145.587135,1145.587135,1018.299676,954.655946,827.368486
1,AIM/CGE 2.0,SSP1-26,AGO,RIME|sdii|Exposure|Land area,km2,0.000000,0.000000,732.186734,1568.971572,1882.765886,1882.765886,1673.569677,1568.971572,1359.775362
2,AIM/CGE 2.0,SSP1-26,ALB,RIME|sdii|Exposure|Land area,km2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,AIM/CGE 2.0,SSP1-26,AND,RIME|sdii|Exposure|Land area,km2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,AIM/CGE 2.0,SSP1-26,ARE,RIME|sdii|Exposure|Land area,km2,18898.082445,8425.001025,1839.398395,2292.485996,2462.393846,2462.393846,2349.121946,2292.485996,2179.214096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1351,WITCH-GLOBIOM 4.4,CD-LINKS_NoPolicy,YEM,RIME|sdii|Hazard|Risk score|Population weighted,risk score,0.127871,0.190785,0.232719,0.192419,0.288064,0.338743,0.326223,0.210727,0.191489
1352,WITCH-GLOBIOM 4.4,CD-LINKS_NoPolicy,ZAF,RIME|sdii|Hazard|Risk score|Population weighted,risk score,0.281809,0.258369,0.229794,0.211687,0.278753,0.278546,0.228141,0.200223,0.195988
1353,WITCH-GLOBIOM 4.4,CD-LINKS_NoPolicy,ZMB,RIME|sdii|Hazard|Risk score|Population weighted,risk score,0.012770,0.012099,0.011437,0.011576,0.026542,0.032043,0.063557,0.211631,0.236261
1354,WITCH-GLOBIOM 4.4,CD-LINKS_NoPolicy,ZWE,RIME|sdii|Hazard|Risk score|Population weighted,risk score,0.012558,0.008738,0.006224,0.008528,0.042863,0.065008,0.078111,0.085855,0.087229


In [ ]:
# expandeddGMT